# WSI-mIF Fine Registration 

This notebook demonstrates the complete workflow for Whole Slide Image (WSI) registration using rigid and non-rigid techniques with nuclei-based analysis.

## Overview
- **Rigid Registration**: Initial coarse alignment using traditional techniques
- **Nuclei Detection**: Extract nuclei from tissue patches
- **Shape-aware Nuclei based Registration**: Point cloud-based fine alignment
- **Non-rigid Registration**: Coherent Point Drift (CPD) for local deformation
- **Interactive Visualization**: Bokeh plots for analysis


## 1. Setup and Imports

In [ ]:
import sys
import os

# Add project root to PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # adjust if needed
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [ ]:
# Enable inline plotting and auto-reload for development
%matplotlib inline
%load_ext autoreload
%autoreload 2
import SimpleITK as sitk
# Import all necessary modules
from wsi_mif_registration.utils.imports import *
from wsi_mif_registration.config import *
from wsi_mif_registration.preprocessing.padding import *
from wsi_mif_registration.preprocessing.preprocessing import *
from wsi_mif_registration.registration.registration import *
from wsi_mif_registration.evaluation.evaluation import *
from wsi_mif_registration.visualization.visualization import *
from wsi_mif_registration.preprocessing.nuclei_analysis import *
from wsi_mif_registration.preprocessing.stainnorm import *
import wsi_mif_registration.registration.nonrigid
# Setup Bokeh for notebook output
setup_bokeh_notebook()

print("✅ All modules imported successfully!")
print(f"Source WSI: {SOURCE_WSI_PATH}")
print(f"Target WSI: {TARGET_WSI_PATH}")

## 2. Configuration Check

Verify that all file paths are correct and files exist.

In [ ]:
import os

# Check if files exist
files_to_check = [
    SOURCE_WSI_PATH,
    TARGET_WSI_PATH,
    FIXED_POINTS_PATH,
    MOVING_POINTS_PATH
]

print("File existence check:")
for file_path in files_to_check:
    exists = os.path.exists(file_path)
    status = "✅" if exists else "❌"
    print(f"{status} {file_path}")

# Display current parameters
print("\nCurrent Parameters:")
print(f"- Preprocessing Resolution: {PREPROCESSING_RESOLUTION}")
print(f"- Registration Resolution: {REGISTRATION_RESOLUTION}")
print(f"- Patch Size: {PATCH_SIZE}")
print(f"- Fixed Threshold: {FIXED_THRESHOLD}")
print(f"- Moving Threshold: {MOVING_THRESHOLD}")
print(f"- Min Nuclei Area: {MIN_NUCLEI_AREA}")

## 3. Load and Preprocess WSI Images

In [ ]:
# Load WSI images
print("Loading WSI images...")
source_wsi, target_wsi, source, target = load_wsi_images(
    SOURCE_WSI_PATH, TARGET_WSI_PATH, PREPROCESSING_RESOLUTION
)

print(f"\nLoaded images:")
print(f"Source shape: {source.shape}")
print(f"Target shape: {target.shape}")

In [ ]:
# Preprocess images
print("Preprocessing images...")
# source_prep,target_prep=source,target
source_prep, target_prep,padding_params =pad_images(source, target)

# Extract tissue masks
print("Extracting tissue masks...")
source_mask, target_mask = extract_tissue_masks(source_prep, target_prep)

print("✅ Preprocessing completed!")

## 4. Visualize Original Images

In [ ]:
# Display original images side by side
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].imshow(source_prep)
axes[0, 0].set_title('Source Image (Moving)')
axes[0, 0].axis('off')

axes[0, 1].imshow(target_prep)
axes[0, 1].set_title('Target Image (Fixed)')
axes[0, 1].axis('off')

axes[1, 0].imshow(source_mask, cmap='gray')
axes[1, 0].set_title('Source Tissue Mask')
axes[1, 0].axis('off')

axes[1, 1].imshow(target_mask, cmap='gray')
axes[1, 1].set_title('Target Tissue Mask')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 5. Perform Rigid Registration

In [ ]:
# Perform rigid registration
print("Performing rigid registration...")
moving_img_transformed, final_transform = perform_rigid_registration(
    source_prep, target_prep, source_mask, target_mask
)

print("\nRigid transformation matrix:")
print(final_transform)

# Visualize transformed image
plt.figure(figsize=(12, 8))
plt.imshow(moving_img_transformed)
plt.title('Rigid Transformed Source Image')
plt.axis('off')
plt.show()



print("✅ Rigid registration completed!")

In [ ]:
registered=cv2.warpAffine(source_prep, final_transform[0:-1],source_prep.shape[:2][::-1] )

In [ ]:
overlay_before=np.dstack((color.rgb2gray(source_prep), color.rgb2gray(target_prep), color.rgb2gray(source_prep)))
overlay_after=np.dstack((color.rgb2gray(registered), color.rgb2gray(target_prep), color.rgb2gray(registered)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left: Transformed image
axes[0].imshow(overlay_before)

axes[0].axis('off')

# Right: RG overlay
axes[1].imshow(overlay_after)

axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
u_x, u_y = util.rigid_dot(source_prep,np.linalg.inv(final_transform))

deformation_field = np.stack(( u_x, u_y), axis=-1)
# deformation_field=util.create_displacement_field_for_wsi(final_transform, source, target)
sitk_image = sitk.GetImageFromArray(deformation_field)
sitk.WriteImage(sitk_image, '/home/u5552013/Nextcloud/hyreco_vis/overlays/533_rigid.mha')

## 6. Perform Non Rigid 

In [ ]:
displacement_field,warped_source= nonrigid.elastic_image_registration(
   moving_img_transformed,
   target_prep,
    similarity_metric_params={'win_size': 7}, 
     compute_device='cuda'  # Use the same device
)
print("non rigid displacement field",displacement_field.shape)

In [ ]:
disp_field_np=util.tc_df_to_np_df(displacement_field)
w_x,w_y=util.compose_vector_fields(u_x, u_y, disp_field_np[0], disp_field_np[1])

deformation_field = np.stack(( w_x, w_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)
# sitk.WriteImage(sitk_image, '/home/u5552013/Nextcloud/hyreco_vis/overlays/533_nonrigid.mha')

## 7. TIA Registration Visualization 

In [ ]:
%%bash 
export BOKEH_ALLOW_WS_ORIGIN=localhost:5007

tiatoolbox visualize --slides "path-to-slides" --overlays "path-to-overlays"


## 8. Scale Transformation and Extract Patches

In [ ]:
# Scale transformation for high resolution analysis
transform_40x = scale_transformation_matrix(
    final_transform, PREPROCESSING_RESOLUTION, REGISTRATION_RESOLUTION
)

print("Scaled transformation matrix (40x):")
print(transform_40x)

# Extract patches from target WSI
print("\nExtracting patches...")
fixed_patch_extractor = extract_patches_from_wsi(
    target_wsi, target_mask, PATCH_SIZE, PATCH_STRIDE
)

print(f"Total patches extracted: {len(fixed_patch_extractor)}")

## 9. Visualize Sample Patches

In [ ]:
# Select a patch for visualization
patch_idx = 70  # You can change this index
loc = fixed_patch_extractor.coordinate_list[patch_idx]
location = (loc[0], loc[1])

print(f"Visualizing patch {patch_idx} at location {location}")

# Extract regions for comparison
fixed_tile = target_wsi.read_rect(location, VISUALIZATION_SIZE, resolution=40, units="power")
moving_tile = source_wsi.read_rect(location, VISUALIZATION_SIZE, resolution=40, units="power")

# Create transformer and extract transformed tile
tfm = AffineWSITransformer(source_wsi, transform_40x)
transformed_tile = tfm.read_rect(location=location, size=VISUALIZATION_SIZE, resolution=0, units="level")

# Visualize patches
visualize_patches(fixed_tile, moving_tile, transformed_tile)

## 10. Nuclei Detection and Analysis

In [ ]:
tmp_dir="path-to-save nuclei-detections"
FIXED_NUCLEI_CSV = os.path.join(tmp_dir, "fixed_nuclei.csv")
MOVING_NUCLEI_CSV = os.path.join(tmp_dir, "moving_nuclei.csv")

In [ ]:
import os

# Define tmp directory and CSV paths
tmp_dir = os.path.join(os.getcwd(), "tmp")
os.makedirs(tmp_dir, exist_ok=True)

FIXED_NUCLEI_CSV = os.path.join(tmp_dir, "fixed_nuclei.csv")
MOVING_NUCLEI_CSV = os.path.join(tmp_dir, "moving_nuclei.csv")

# Process nuclei detection on all patches
print("Processing nuclei detection on all patches...")
print("This may take several minutes depending on the number of patches...")
print("\n💡 Based on the validation above, the detection parameters look good!")

# Process a subset of patches for demonstration (adjust range as needed)
start_idx = 0
end_idx = max(50, len(fixed_patch_extractor) - 1)  # Process first 50 patches

print(f"Processing patches {start_idx} to {end_idx} ({end_idx-start_idx+1} patches)")
print("For full analysis, set end_idx = len(fixed_patch_extractor) - 1")

all_fixed_nuclei_data, all_moving_nuclei_data = process_nuclei_in_patches(
    fixed_patch_extractor, tfm, start_idx, end_idx
)

print(f"\n🔬 Nuclei Detection Results:")
print(f"- Fixed nuclei detected: {len(all_fixed_nuclei_data)}")
print(f"- Moving nuclei detected: {len(all_moving_nuclei_data)}")
print(f"- Average nuclei per patch:")
print(f"  * Fixed: {len(all_fixed_nuclei_data)/(end_idx-start_idx+1):.1f}")
print(f"  * Moving: {len(all_moving_nuclei_data)/(end_idx-start_idx+1):.1f}")

# Save nuclei data into tmp/
save_nuclei_data_to_csv(
    all_fixed_nuclei_data, all_moving_nuclei_data,
    FIXED_NUCLEI_CSV, MOVING_NUCLEI_CSV
)

print(f"\n✅ Nuclei CSVs saved:")
print(f"- Fixed nuclei: {FIXED_NUCLEI_CSV}")
print(f"- Moving nuclei: {MOVING_NUCLEI_CSV}")

## 10. Interactive Nuclei Visualization

In [ ]:
# Load nuclei coordinates
moving_df = load_nuclei_coordinates(MOVING_NUCLEI_CSV)
fixed_df = load_nuclei_coordinates(FIXED_NUCLEI_CSV)

print(f"Loaded nuclei data:")
print(f"- Fixed nuclei: {len(fixed_df)}")
print(f"- Moving nuclei: {len(moving_df)}")

# Create basic nuclei overlay plot
print("\nCreating interactive nuclei overlay plot...")
plot1 = create_nuclei_overlay_plot(moving_df, fixed_df, 
                                  "Original Nuclei Coordinates (Before Registration)")
show_plot(plot1)

## 11. Shape-Aware Point Set Registration

In [ ]:
# Perform shape-aware registration as an alternative to ICP
print("Performing Shape-Aware Point Set Registration...")

shape_registrator,shape_transform, shape_transformed_coords   = perform_shape_aware_registration(
     fixed_df,moving_df,
    shape_attribute='area',
    shape_weight=0.1,  # 30% weight for shape, 70% for spatial distance
    max_iterations=200,
    tolerance=1e-11
)

moving_df_shape_aware = create_nuclei_dataframe_from_points(shape_transformed_coords)
# Add registered coordinates to the original dataframe for visualization
moving_df_shape_aware = moving_df.copy()
moving_df_shape_aware['global_x'] = shape_transformed_coords[:, 0]
moving_df_shape_aware['global_y'] = shape_transformed_coords[:, 1]

print("✅ Shape-aware registration completed!")


In [ ]:
# translation_matrix, rotation_scale_matrix
scale_factor = 0.625 / 40  # 0.25
transform_3x3_scaled = shape_transform.copy()
transform_3x3_scaled[0:2, 2] = transform_3x3_scaled[0:2, 2] * scale_factor
sh_transform=transform_3x3_scaled
shape_fine_transform=sh_transform@final_transform.copy()

In [ ]:
u_x, u_y = util.rigid_dot(target_prep, np.linalg.inv(shape_fine_transform))

deformation_field = np.stack(( u_x, u_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)
# sitk.WriteImage(sitk_image, '/home/u5552013/Nextcloud/hyreco/overlays/679_finerigid_wop.mha')

In [ ]:
%%bash 
export BOKEH_ALLOW_WS_ORIGIN=localhost:5007

tiatoolbox visualize --slides "/home/u5552013/Nextcloud/hyreco/slides" --overlays "/home/u5552013/Nextcloud/hyreco/overlays"


## 11.1. Visualize Shape-Aware Registration Details

In [ ]:
# Create detailed shape-aware registration visualization
print("Creating detailed shape-aware registration visualization...")

# Use the built-in visualization method of the shape-aware registrator
plot_shape_aware = visualize_shape_aware_registration(
    shape_registrator, 
    title=f"Shape-Aware Registration (Weight: {shape_registrator.shape_weight})"
)
show_plot(plot_shape_aware)

# Show parameter analysis
print(f"\n🔍 Shape-Aware Registration Analysis:")
print(f"- Shape attribute used: {shape_registrator.shape_attribute}")
print(f"- Shape weight: {shape_registrator.shape_weight} (importance of shape vs spatial distance)")
print(f"- Spatial weight: {1 - shape_registrator.shape_weight}")
print(f"- Correspondences found: {len(shape_registrator.correspondence_indices)}")
print(f"- Optimization method: Powell with {shape_registrator.max_iterations} max iterations")

# Analyze shape attribute distribution
fixed_shape_stats = shape_registrator.fixed_points['area'].describe()
moving_shape_stats = shape_registrator.moving_points['area'].describe()

print(f"\n📊 Shape Attribute (Area) Statistics:")
print(f"Fixed nuclei area - Mean: {fixed_shape_stats['mean']:.1f}, Std: {fixed_shape_stats['std']:.1f}")
print(f"Moving nuclei area - Mean: {moving_shape_stats['mean']:.1f}, Std: {moving_shape_stats['std']:.1f}")

# Suggest parameter tuning
print(f"\n💡 Parameter Tuning Suggestions:")
if shape_registrator.final_error > 50:
    print("- Consider increasing shape_weight if nuclei areas are well-preserved between images")
    print("- Consider decreasing shape_weight if there's significant area variation")
else:
    print("- Current parameters seem well-tuned for this dataset")

if abs(fixed_shape_stats['mean'] - moving_shape_stats['mean']) > 50:
    print("- Large difference in mean areas detected - shape weight might need adjustment")
else:
    print("- Area distributions are similar between datasets - good for shape-aware registration")

In [ ]:
from sklearn.cluster import KMeans
import numpy as np


moving_df_shape_aware = moving_df.copy()
moving_df_shape_aware['global_x'] = shape_transformed_coords[:, 0]
moving_df_shape_aware['global_y'] = shape_transformed_coords[:, 1]
transformed_points_nuclei=moving_df_shape_aware[['global_x', 'global_y']]
transformed_points_nuclei = transformed_points_nuclei[['global_x', 'global_y']].to_numpy()
fixed_points_nuclei = fixed_df[['global_x', 'global_y']].to_numpy()

# Find mutual nearest neighbors for robust correspondence
print("Finding mutual nearest neighbors...")
fixed_mnn, moving_mnn, mnn_pairs = find_mutual_nearest_neighbors(
    fixed_points_nuclei,transformed_points_nuclei
)
N_CLUSTERS = 1000  # Try 500–2000 depending on available RAM

kmeans_fixed = KMeans(n_clusters=N_CLUSTERS, random_state=42).fit(fixed_mnn)
kmeans_moving = KMeans(n_clusters=N_CLUSTERS, random_state=42).fit(moving_mnn)

fixed_centers = kmeans_fixed.cluster_centers_
moving_centers = kmeans_moving.cluster_centers_

print(f"MNN pairs found: {len(mnn_pairs)}")

# Perform CPD non-rigid registration
print("\nPerforming CPD non-rigid registration...")
print("This may take a few minutes...")

moving_nonrigid_transformed = perform_cpd_registration(
    moving_centers, fixed_centers,
    beta=RegistrationParams.CPD_BETA,
    alpha=RegistrationParams.CPD_ALPHA,
    max_iterations=RegistrationParams.CPD_MAX_ITERATIONS,
    tolerance=RegistrationParams.CPD_TOLERANCE
)

print("✅ CPD registration completed!")


## 13. Visualize Registration Comparison

In [ ]:
# Create DataFrames for visualization
moving_df_nonrigid = create_nuclei_dataframe_from_points(moving_nonrigid_transformed)
fixed_df_mnn = create_nuclei_dataframe_from_points(fixed_centers)
moving_df_mnn = create_nuclei_dataframe_from_points(moving_centers)



plot3 = create_registration_comparison_plot(
    fixed_df_mnn, moving_df_mnn, moving_df_mnn, moving_df_nonrigid
)
show_plot(plot3)



## 14. Generate Displacement Field

In [ ]:
# Create displacement field for spatial transformation analysis
print("Creating displacement field...")

# Scale down coordinates for field generation
scale_factor = 64
source_points_scaled = moving_centers  / scale_factor
target_points_scaled = moving_nonrigid_transformed / scale_factor

displacement_field = create_displacement_field(
    source_points_scaled, target_points_scaled,
    target.shape,
    method=RegistrationParams.INTERPOLATION_METHOD,
    sigma=RegistrationParams.DISPLACEMENT_SIGMA,
    max_displacement=RegistrationParams.MAX_DISPLACEMENT
)


In [ ]:
from scipy.ndimage import map_coordinates
import numpy as np
import pandas as pd

# Convert coordinates from microns to pixels
moving_points_px = transformed_points_sorted / 16
fixed_points_px = fixed_points_sorted / 16

# Image shape
H, W, _ = displacement_field.shape

# Prepare coordinates
x_coords = moving_points_px[:, 0]
y_coords = moving_points_px[:, 1]

# Validity mask
valid_mask = (x_coords >= 0) & (x_coords < W) & (y_coords >= 0) & (y_coords < H)

# Filter valid
x_coords_valid = x_coords[valid_mask]
y_coords_valid = y_coords[valid_mask]
fixed_points_valid = fixed_points_px[valid_mask]

# Stack coordinates and normalize for coarse displacement field
sample_coords = np.vstack([y_coords_valid, x_coords_valid])   # shape: (2, N)

# Interpolate displacements from field (downsampled by 64)
dx_interp = map_coordinates(displacement_field[:, :, 0] / 64, sample_coords, order=1)
dy_interp = map_coordinates(displacement_field[:, :, 1] / 64, sample_coords, order=1)
print(dx_interp, dy_interp)
# Apply interpolated displacements
moved_points = np.vstack([
    x_coords_valid + dx_interp ,
    y_coords_valid + dy_interp
]).T

w_x,w_y=util.compose_vector_fields(u_x, u_y, displacement_field[0], displacement_field[1])

deformation_field = np.stack(( w_x, w_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)
# sitk.WriteImage(sitk_image, '/home/u5552013/Nextcloud/hyreco_vis/overlays/533_nonrigid.mha')


## 15. TIA Registration Visualization

In [ ]:
%%bash 
export BOKEH_ALLOW_WS_ORIGIN=localhost:5007

tiatoolbox visualize --slides "path-to-slides" --overlays "path-to-overlays"
